### Importing Data

In [3]:
from xml.dom import minidom
import pandas as pd

In [4]:
# Reading xml dataset
xmldoc = minidom.parse('/content/dataxml.xml')
sentences = xmldoc.getElementsByTagName("sentence")
df = pd.DataFrame()
sentence_list = []
aspect_list = []
polarity_list = []
for sentence in sentences:
  for i in range(len(sentence.getElementsByTagName("aspectTerm"))):
    sentence_list.append(sentence.getElementsByTagName("text")[0].firstChild.data)
    aspect_list.append(sentence.getElementsByTagName("aspectTerm")[i].getAttribute('term'))
    polarity_list.append(sentence.getElementsByTagName("aspectTerm")[i].getAttribute('polarity'))

In [5]:
# Forming DataFrame
df['sentence'] = sentence_list
df['aspect'] = aspect_list
df['polarity'] = polarity_list

### Installing the Hugging Face Library

In [6]:
!pip install transformers

     |████████████████████████████████| 1.3MB 5.5MB/s 
     |████████████████████████████████| 2.9MB 19.4MB/s 
     |████████████████████████████████| 890kB 41.3MB/s 
     |████████████████████████████████| 1.1MB 37.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9c55629740d23a426305feca68fb8df6143099fe954e0b375334398a4e49e5d8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### Limit comments length

In [7]:
from transformers import AutoTokenizer
# Using Pars-BERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")

# Using Multilingual BERT Tokenizer
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [8]:
# Calculate the number of tokens in each comment
df['len_sentence'] = df['sentence'].apply(lambda t: len(tokenizer.tokenize(t)))
print(df['len_sentence'].max())
print(df['len_sentence'].min())

1248
2


In [9]:
# Finding the share of comments with a specific length
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='len_sentence'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [10]:
data_gl_than(df, 256, 1)

Texts with word length of greater than 1 and less than 256 includes 97.51% of the whole!


In [11]:
# Set min and max limits of length
minlim, maxlim = 1, 256

In [12]:
# remove comments with the length of fewer than three words
df['len_sentence'] = df['len_sentence'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
df = df.dropna(subset=['len_sentence'])
df = df.reset_index(drop=True)

In [13]:
print(df['len_sentence'].max())
print(df['len_sentence'].min())

254.0
2.0


In [14]:
df = df.drop(columns='len_sentence')

### Train-Test split

In [15]:
# Create a 85-15 train-test split.
import numpy as np
df_train, df_test = np.split(df.sample(frac=1, random_state=4040), [int(.85*len(df))])
print('{:>5,} training samples'.format(len(df_train)))
print('{:>5,} test samples'.format(len(df_test)))

8,290 training samples
1,463 test samples


### Train data preparation

In [16]:
df_train['polarity'] = df_train['polarity'].replace(['positive', 'negative', 'neutral'], [0, 1, 2])
df_train = df_train.reset_index(drop=True)

comments_a = df_train['sentence'].tolist()
comments_b = df_train['aspect'].tolist()
labels = df_train['polarity'].tolist()

num_labels = 3

print(df_train.shape)
df_train.head(5)

(8290, 3)


,sentence,aspect,polarity
0,من اینو حدودا دوماهی هست گرفتم. اینترنت نامحدو...,سرعت دانلود,0
1,دنبال یه تبلت برند با قیمت ارزان بودم که اینو ...,کیفیت اسپیکر,2
2,ایرفون بسیار خوش ساخت و با کیفیتی هستش و ازش ک...,وان مور,0
3,شمابه چی این دلتونو خوش کردین نه یک باتری درست...,باطری,0
4,من از روسیه این رو خریدم که ۱۰۰ میل شد حدود ۹۰...,روسیه,2


### Using Colab GPU for Training

In [17]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


### Encoding the data

In [18]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
token_type_ids = []

# For every sentence pair...
for (comment_a, comment_b) in zip(comments_a, comments_b):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        comment_a,           # Sentence to encode.
                        comment_b,           # Sentence to encode.
                        truncation = True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 270,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_token_type_ids = True,
                   )
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

    # And its token_type_ids    
    token_type_ids.append(encoded_dict['token_type_ids'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)


# Print sentence 0, now as a list of IDs.
print('comment_a: ', comments_a[0])
print('comment_b: ', comments_b[0])
print('Token IDs:', input_ids[0])
print('Token Type IDs:', token_type_ids[0])
print('Token Attention Mask:', attention_masks[0])

comment_a:  من اینو حدودا دوماهی هست گرفتم. اینترنت نامحدود همراه اول گرفتم از سایت ۱ شب تا ۱۱ صبح خونه که هستم سرعت دانلود تا ۵ مگ میرسه و سرکارم که میارم اینجا ۴٫۵ هست، سرعت دانلود تا ۱۴ مگ میرسه خیلی راحت. البته داخل خودتنظیمات مودم بری نوشته ۴G ولی اینجایی که من هستم ۴٫۵G هست و سرعتش فوق العاده عالیه. برنامه موبایلشو هم نصب کنید به راحتی میتونید از طریق خود برنامش شارژ کنید سیم کارتتون. بعضی از دوستان نوشته بودن داغ میکنه، ولی واسه من اصلا داغ نمیکنه به هیچ وج اکثرا هم من به پاور بانک میزنم استفاده میکنم 
comment_b:  سرعت دانلود
Token IDs: tensor([    2,  2842, 25021, 10563, 32848,  2003,  2952, 11038,  1012,  4900,
        13887,  3287,  3067, 11038,  2791,  4394,  1455,  3256,  2848,  3797,
         5072, 19328,  2800,  6794,  4089,  7510,  2848,  1459,  6333, 34885,
         1379, 20785,  2015,  2800, 82434,  4938,  1458,  1393,  1459,  2952,
         1348,  4089,  7510,  2848,  4012,  6333, 34885,  3805,  6309,  1012,
         3546,  4127, 14587,  7380,  7786,  2009, 17157,  79

### Import the BERT model

In [19]:
from transformers import AutoModelForSequenceClassification, AutoConfig, AdamW
# Using Pars-BERT model
model = AutoModelForSequenceClassification.from_pretrained(
    "HooshvareLab/bert-fa-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = num_labels, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    hidden_dropout_prob=0.1, 
)
config = AutoConfig.from_pretrained("HooshvareLab/bert-fa-base-uncased")

# Using Multilingual BERT model
# model = AutoModelForSequenceClassification.from_pretrained(
#     "bert-base-multilingual-cased", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = num_labels, # The number of output labels--2 for binary classification.
#                     # You can increase this for multi-class tasks.   
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = False, # Whether the model returns all hidden-states.
#     hidden_dropout_prob=0.1, 
# )
# config = AutoConfig.from_pretrained("bert-base-multilingual-cased")

# Tell pytorch to run this model on the GPU.
if device.type == 'cuda':
  model.cuda()

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

### Combine the training inputs into a TensorDataset

In [20]:
from torch.utils.data import TensorDataset
train_dataset = TensorDataset(input_ids, token_type_ids, attention_masks, labels)
print('{:>5,} training samples'.format(len(train_dataset)))

8,290 training samples


### Create dataloaders

In [21]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

### Create Optimizer, Scheduler, Accuracy, Elapsed time

In [22]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [23]:
# Function to calculate accuracy
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
# Functin for elapsed time
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

### Training loop

In [25]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 2020

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: token type ids
        #   [2]: attention masks
        #   [3]: labels 
        b_input_ids = batch[0].to(device)
        b_token_type_ids =  batch[1].to(device)
        b_input_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=b_token_type_ids,
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()
   

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Training Time': training_time,
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    519.    Elapsed: 0:00:56.
  Batch    80  of    519.    Elapsed: 0:01:53.
  Batch   120  of    519.    Elapsed: 0:02:51.
  Batch   160  of    519.    Elapsed: 0:03:48.
  Batch   200  of    519.    Elapsed: 0:04:45.
  Batch   240  of    519.    Elapsed: 0:05:42.
  Batch   280  of    519.    Elapsed: 0:06:39.
  Batch   320  of    519.    Elapsed: 0:07:36.
  Batch   360  of    519.    Elapsed: 0:08:33.
  Batch   400  of    519.    Elapsed: 0:09:30.
  Batch   440  of    519.    Elapsed: 0:10:27.
  Batch   480  of    519.    Elapsed: 0:11:24.

  Average training loss: 0.50
  Training epcoh took: 0:12:19

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    519.    Elapsed: 0:00:57.
  Batch    80  of    519.    Elapsed: 0:01:54.
  Batch   120  of    519.    Elapsed: 0:02:51.
  Batch   160  of    519.    Elapsed: 0:03:48.
  Batch   200  of    519.    Elapsed: 0:04:45.
  Batch   240  of    519.    Elapsed: 0:05:42.
  Batc

### Preparing test data

In [26]:
df_test['polarity'] = df_test['polarity'].replace(['positive', 'negative', 'neutral'], [0, 1, 2])
df_test = df_test.reset_index(drop=True)

comments_a = df_test['sentence'].tolist()
comments_b = df_test['aspect'].tolist()
labels = df_test['polarity'].tolist()


print(df_test.shape)
df_test.head(5)

(1463, 3)


,sentence,aspect,polarity
0,امروز ۳۲G رو بدون گارانتی خریدم و با هات اسپات...,دوربین سلفی,1
1,گوشی باکیفت و عالییه.,گوشی,0
2,من در پیشنهاد شگفت انگیز تهیهاش کردم و امروز ب...,جنس چرم,2
3,من حدودا یه ماهه اینو خریدم و یه شارژر بدون اس...,برد وایرلسشم,0
4,بزرگترین مشکلش برای من اینه زمان شارژش خیلی با...,سخت افزار,2


In [27]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
token_type_ids = []

# For every sentence...
for (comment_a, comment_b) in zip(comments_a, comments_b):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        comment_a,           # Sentence to encode.
                        comment_b,           # Sentence to encode.
                        truncation = True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 270,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_token_type_ids = True,
                   )
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

    # And its token_type_ids    
    token_type_ids.append(encoded_dict['token_type_ids'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [28]:
# Combine the training inputs into a TensorDataset.
test_dataset = TensorDataset(input_ids, token_type_ids, attention_masks, labels)
print('{:>5,} test samples'.format(len(test_dataset)))
# Set the batch size.  
batch_size = 32 
# Create the DataLoader.
prediction_data = test_dataset
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

1,463 test samples


### Model Evaluation on Test Data

In [29]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

total_eval_accuracy=0
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_token_type, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids= b_token_type, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
  total_eval_accuracy += flat_accuracy(logits, label_ids)

avg_val_accuracy = total_eval_accuracy / len(prediction_dataloader)
print("  Accuracy: {0:.3f}".format(avg_val_accuracy))

Predicting labels for 1,463 test sentences...
  Accuracy: 0.903


In [30]:
# Calculating Macro F1
from sklearn import metrics
preds = [j for i in predictions for j in i]
preds_flat = np.argmax(preds, axis=1).flatten()
labels = [j for i in true_labels for j in i]
f1_score_micro = metrics.f1_score(labels, preds_flat, average='macro')
print("  f1_score: {0:.3f}".format(f1_score_micro))

  f1_score: 0.893
